<span style="color:red">
**_Duplicate of hunspell-run-evaluation-get-upper-bound notbook_! <br>
Here: Get upper bound for CASE INSENSITIVE cases; i. e. titlecase all words**


Explanation:
- if verbs are upper case: okay
- if adjectives are upper case: okay
- if nouns are lower case: falsy
</span>    
___
In notebook _hunspell-analysis_ it has been observed that there are target words which are generally NOT in the dictionary of hunspell. Nevertheless hunspell makes a suggestion for corresponding missspelling and the target word does appear in the list of suggestions anyway.

Therefore it has been decided to not compare the target words to the hunspell dictionary to extract the upper bound **but to give all the target words into hunspell and see which ones are going to be predicted correctly**.

The code is merely copied from _hunspell_run_evaluation_ notebook.

In [1]:
# IMPORTS
#%autoreload 2

import re
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from tqdm.notebook import tqdm # A fast, extensible progress bar

import sys
# TODO: Relativen Pfad einbauen
# --> Issue aufmachen oder elegantere Ordnerstruktur

sys.path.insert(0, '/Users/lisaprepens/_Programmierung/bodu-spell/spelling_correction')
#sys.path.insert(0, '../../')
# sys.path

from Lisa import litkey_2
#import Lisa.litkey_2


# CONFIGURATION
# Set figure size for sns plot
%config InlineBackend.figure_format = 'retina'
mpl.rc('figure', figsize=(8, 6), dpi=100)
sns.set()
sns.set_style('darkgrid')

# Set tqdm on pandas
tqdm.pandas(desc="Progress so far...")

# Do not truncate columns of DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

ModuleNotFoundError: No module named 'Lisa'

In [2]:
# LOAD LITKEY DATA SET
# data sets are read in CASE SENSITIVE and without dropping duplicates on tupel base, but in the next step seperately
data_error_token = litkey_2.load(litkey_data_path ='../../litkey-data/', toss_duplicates=False)
data_error_types = litkey_2.load(litkey_data_path ='../../litkey-data/')

In [3]:
# GENERATE TYPES WITH SUBSET "CORRECTED"
# Method see statistics_litkey_corpus notebook
# data_error_types = data_error_token.drop_duplicates(subset=['original', 'corrected'])
# data_error_types = data_error_types.drop_duplicates(subset='corrected')
# display(data_error_types.shape[0])

In [4]:
# GENERATE TYPES WITH SUBSET "CORRECTED"
# data_error_typesxx = data_error_token.drop_duplicates(subset='corrected')
# data_error_types.reset_index(drop=True, inplace=True)
# display(data_error_typesxx.shape[0])
# Token: Is okay

In [5]:
# data_error_types.head(100)

In [6]:
# PREPROCESSING
#Hunspell treats words with some punctuation marks as two words (e.g. Seil=bahn), which would destroy the indices; 
#therefore, "=" is replaced with "-" and the others are deleted prior to analysis with Hunspell

data_error_types['unchanged_corrected'] = data_error_types['corrected']
data_error_types['corrected'] = data_error_types['corrected'].str.replace("=", '-')
data_error_types['corrected'] = data_error_types['corrected'].str.replace('"', '')
data_error_types['corrected'] = data_error_types['corrected'].str.replace(":", '')
data_error_types['corrected'] = data_error_types['corrected'].str.replace(",", '')
data_error_types['corrected'] = data_error_types['corrected'].str.replace("'", '')

data_error_token['unchanged_corrected'] = data_error_token['corrected']
data_error_token['corrected'] = data_error_token['corrected'].str.replace("=", '-')
data_error_token['corrected'] = data_error_token['corrected'].str.replace('"', '')
data_error_token['corrected'] = data_error_token['corrected'].str.replace(":", '')
data_error_token['corrected'] = data_error_token['corrected'].str.replace(",", '')
data_error_token['corrected'] = data_error_token['corrected'].str.replace("'", '')

In [7]:
# TITLE CASE (original, corrected); Can be directly applied to Series.str
data_error_types.original = data_error_types.original.str.title()
data_error_types.corrected = data_error_types.corrected.str.title()

data_error_token.original = data_error_token.original.str.title()
data_error_token.corrected = data_error_token.corrected.str.title()

In [9]:
display(data_error_types.shape[0])
display(data_error_types.head(10))
display(data_error_types.tail(10))

display(data_error_token.head(10))
display(data_error_token.tail(10))

9484

,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_corrected
0,Belt,Bellt,01-005-2-III-Eis.csv,91,138,91,bellt
1,Kukt,Kuckt,01-005-2-III-Eis.csv,73,152,73,kuckt
2,Dan,Dann,01-005-2-III-Eis.csv,627,651,621,dann
3,Gekricht,Gekriegt,01-005-2-III-Eis.csv,2,15,2,gekriegt
4,Leken,Lecken,01-005-2-III-Eis.csv,14,17,14,lecken
5,Felt,Fällt,01-005-2-III-Eis.csv,93,198,90,fällt
6,Wolte,Wollte,01-005-2-III-Eis.csv,173,201,173,wollte
7,Lekt,Leckt,01-005-2-III-Eis.csv,20,42,19,leckt
8,Fom,Vom,01-005-2-III-Eis.csv,13,16,13,vom
9,Gawen,Kaufen,01-006-2-III-Eis.csv,1,6,1,kaufen


,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_corrected
9474,Hette,Hätte,10-693-4-IV-Weg.csv,1,5,1,hätte
9475,Telefen,Telefon,10-693-4-IV-Weg.csv,1,39,1,Telefon
9476,Aufzumunter,Aufzumuntern,10-693-4-IV-Weg.csv,1,1,1,aufzumuntern
9477,Ungefär,Ungefähr,10-693-4-IV-Weg.csv,1,1,1,Ungefähr
9478,Nooh,Noch,10-693-4-IV-Weg.csv,1,15,1,noch
9479,Nieht,Nicht,10-693-4-IV-Weg.csv,2,135,2,nicht
9480,Vessen,Fassen,10-693-4-IV-Weg.csv,1,2,1,fassen
9481,Tor,Tür,10-693-4-IV-Weg.csv,1,15,1,Tür
9482,Mutzte,Nutzte,10-693-4-IV-Weg.csv,1,1,1,nutzte
9483,Momen,Moment,10-693-4-IV-Weg.csv,1,4,1,Moment


,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_corrected
0,Belt,Bellt,01-005-2-III-Eis.csv,91,138,91,bellt
1,Kukt,Kuckt,01-005-2-III-Eis.csv,73,152,73,kuckt
2,Dan,Dann,01-005-2-III-Eis.csv,627,651,621,dann
3,Gekricht,Gekriegt,01-005-2-III-Eis.csv,2,15,2,gekriegt
4,Leken,Lecken,01-005-2-III-Eis.csv,14,17,14,lecken
5,Felt,Fällt,01-005-2-III-Eis.csv,93,198,90,fällt
6,Wolte,Wollte,01-005-2-III-Eis.csv,173,201,173,wollte
7,Lekt,Leckt,01-005-2-III-Eis.csv,20,42,19,leckt
8,Fom,Vom,01-005-2-III-Eis.csv,13,16,13,vom
9,Dan,Dann,01-005-2-III-Eis.csv,627,651,621,dann


,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_corrected
24591,Nieht,Nicht,10-693-4-IV-Weg.csv,2,135,2,nicht
24592,Vessen,Fassen,10-693-4-IV-Weg.csv,1,2,1,fassen
24593,Entlich,Endlich,10-693-4-IV-Weg.csv,10,20,10,endlich
24594,Tor,Tür,10-693-4-IV-Weg.csv,1,15,1,Tür
24595,Umarte,Umarmte,10-693-4-IV-Weg.csv,2,16,2,umarmte
24596,Nieht,Nicht,10-693-4-IV-Weg.csv,2,135,2,nicht
24597,Ass,Aß,10-693-4-IV-Weg.csv,3,7,3,aß
24598,Mutzte,Nutzte,10-693-4-IV-Weg.csv,1,1,1,nutzte
24599,Momen,Moment,10-693-4-IV-Weg.csv,1,4,1,Moment
24600,Lutscha,Lutscher,10-693-4-IV-Weg.csv,8,56,8,Lutscher


### a) Types

In [10]:
# RUN HUNSPELL
# Create csv-file containing 'CORRECTED'-column (target) data to be analysed with hunspell
data_error_types.corrected.to_csv('corrected_hun_titlecase.csv', header=False, index=False)

# The beforementioned csv-file is input to hunspell analysis with german dictionary;
# Output of analysis is put to a txt file
# Build a file 'hunspell.txt' with corrections for each word
!cat corrected_hun_titlecase.csv | hunspell -d de_DE > output_corrected_hun_titlecase.txt
#!type original_hun.csv | hunspell -d de_DE > hunspell.txt



# Build a list (suggestions-column) of top correction suggestions from hunspell for each word, '' for correct, '?' for unrecognized
# Reading in all the lines
hs = []
with open('output_corrected_hun_titlecase.txt') as f:
    next(f)
    for index, l in enumerate([l for l in [line.strip() for line in f] if l]):
        # Words recognized as correct
        # (* = dictionary stem (e. g. "man"), + = affixed forms of the following dictionary stem (e. g. "wollt" - wollen))
        # Append nothing to hs
        
        # TODO: Why minus flag?
        if re.match(r'\+|\*|-', l):
            hs.append('')
            
        # Words not recognized/rejected words (# = without suggestions)
        # Append '?' to hs
        elif re.match('#', l):
            hs.append('?')
            
        # Words not recognized/rejected words (& = with suggestions)
        # Append suggested words to hs
        else:
            hs.append(l.split(': ')[1].split(', '))

            
# Add hunspell's corrections as column to data
print(data_error_types.shape[0])
print(len(hs))

data_error_types['suggestions'] = hs

data_error_types.head(40)    # Print excerpt from DataFrame

9484
9484


,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_corrected,suggestions
0,Belt,Bellt,01-005-2-III-Eis.csv,91,138,91,bellt,
1,Kukt,Kuckt,01-005-2-III-Eis.csv,73,152,73,kuckt,"[Juckt, Duckt, Guckt, Zuckt, Kickt, Kuckuck]"
2,Dan,Dann,01-005-2-III-Eis.csv,627,651,621,dann,
3,Gekricht,Gekriegt,01-005-2-III-Eis.csv,2,15,2,gekriegt,
4,Leken,Lecken,01-005-2-III-Eis.csv,14,17,14,lecken,
5,Felt,Fällt,01-005-2-III-Eis.csv,93,198,90,fällt,
6,Wolte,Wollte,01-005-2-III-Eis.csv,173,201,173,wollte,
7,Lekt,Leckt,01-005-2-III-Eis.csv,20,42,19,leckt,
8,Fom,Vom,01-005-2-III-Eis.csv,13,16,13,vom,
9,Gawen,Kaufen,01-006-2-III-Eis.csv,1,6,1,kaufen,


### b) Token

In [11]:
# Create csv-file containing 'CORRECTED'-column (target) data to be analysed with hunspell
data_error_token.corrected.to_csv('corrected_hun_token_titlecase.csv', header=False, index=False)

# The beforementioned csv-file is input to hunspell analysis with german dictionary;
# Output of analysis is put to a txt file
# Build a file 'hunspell.txt' with corrections for each word
!cat corrected_hun_token_titlecase.csv | hunspell -d de_DE > output_corrected_hun_token_titlecase.txt
#!type original_hun.csv | hunspell -d de_DE > hunspell.txt



# Build a list (suggestions-column) of top correction suggestions from hunspell for each word, '' for correct, '?' for unrecognized
# Reading in all the lines
hs = []
with open('output_corrected_hun_token_titlecase.txt') as f:
    next(f)
    for index, l in enumerate([l for l in [line.strip() for line in f] if l]):
        # Words recognized as correct
        # (* = dictionary stem (e. g. "man"), + = affixed forms of the following dictionary stem (e. g. "wollt" - wollen))
        # Append nothing to hs
        
        # TODO: Why minus flag?
        if re.match(r'\+|\*|-', l):
            hs.append('')
            
        # Words not recognized/rejected words (# = without suggestions)
        # Append '?' to hs
        elif re.match('#', l):
            hs.append('?')
            
        # Words not recognized/rejected words (& = with suggestions)
        # Append suggested words to hs
        else:
            hs.append(l.split(': ')[1].split(', '))

            
# Add hunspell's corrections as column to data
print(data_error_token.shape[0])
print(len(hs))

data_error_token['suggestions'] = hs

data_error_token.head(40)    # Print excerpt from DataFrame

24601
24601


,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_corrected,suggestions
0,Belt,Bellt,01-005-2-III-Eis.csv,91,138,91,bellt,
1,Kukt,Kuckt,01-005-2-III-Eis.csv,73,152,73,kuckt,"[Juckt, Duckt, Guckt, Zuckt, Kickt, Kuckuck]"
2,Dan,Dann,01-005-2-III-Eis.csv,627,651,621,dann,
3,Gekricht,Gekriegt,01-005-2-III-Eis.csv,2,15,2,gekriegt,
4,Leken,Lecken,01-005-2-III-Eis.csv,14,17,14,lecken,
5,Felt,Fällt,01-005-2-III-Eis.csv,93,198,90,fällt,
6,Wolte,Wollte,01-005-2-III-Eis.csv,173,201,173,wollte,
7,Lekt,Leckt,01-005-2-III-Eis.csv,20,42,19,leckt,
8,Fom,Vom,01-005-2-III-Eis.csv,13,16,13,vom,
9,Dan,Dann,01-005-2-III-Eis.csv,627,651,621,dann,


In [9]:
#data_error_types.head(100)
#data_error_token.head(100)

In [11]:
# Serialize dataframe, so that analysis does not have to be run every single time
# Dataframe can then be deserialized in another notebook
#data_error_types.to_pickle('target_error_types_hunspell_evaluation_titlecase.pkl')
#data_error_token.to_pickle('target_error_token_hunspell_evaluation_titlecase.pkl')

## Get upper bound
_Here: For case insensitive_ <br>
There are 3 cases to be distinguished:
- not recognized
    - Hunspell flag = '#' , i. e. append '?' to hs (list = [?])
- recognized as right
    - Hunspell flag = '+' or '*', i. e. append nothing to hs (see above)
- recognized as wrong
    - list contains suggestions

### a) Types

In [12]:
# Deserialze
data_error_types = pd.read_pickle('target_error_types_hunspell_evaluation_titlecase.pkl')
data_error_token = pd.read_pickle('target_error_token_hunspell_evaluation_titlecase.pkl')

In [9]:
# NOT RECOGNIZED

# Where is "Erdbeerlollipop"?
# Can be found at "erdbeerlollipop" since lowercasing in evaluation function (corrected)
# or in column 'corrected_cs'

#display(data_error_types[data_error_types.corrected == 'erdbeerlollipop'])
#display(data_error_types[data_error_types.corrected_cs == 'Erdbeerlollipop'])

In [3]:
# GET UPPER BOUND FOR TARGET WORDS; I. E. TYPES FOR TARGET
# in data error_types, "types" are on tupel base
data_error_target_types = data_error_types.drop_duplicates(subset='corrected')
display(data_error_target_types.shape[0])

2959

In [4]:
ct_hun_not_right_TT = data_error_target_types[data_error_target_types.suggestions.apply(len).gt(0)].shape[0] # check for length greater than 0, i. e. list not empty
ct_hun_right_TT = data_error_target_types[~data_error_target_types.suggestions.apply(len).gt(0)].shape[0] # not right is either false or not recognized
display('Hunspell recognizes as not correct...', data_error_target_types[data_error_target_types.suggestions.apply(len).gt(0)].head(40))

'Hunspell recognizes as not correct...'

,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_corrected,suggestions
1,Kukt,Kuckt,01-005-2-III-Eis.csv,73,152,73,kuckt,"[Juckt, Duckt, Guckt, Zuckt, Kickt, Kuckuck]"
14,Nahaise,Nachhause,01-006-2-III-Eis.csv,1,10,1,nachhause,"[Nach hause, Nach-hause, Nachschaue, Nachschau]"
28,Weitagegangen,Weitergegangen,01-025-2-III-Eis.csv,1,1,1,weitergegangen,"[Weitergengagen, Weitergegengen, Weiter gegang..."
63,Runtergefalen,Runtergefallen,01-029-2-III-Eis.csv,12,28,12,runtergefallen,"[-untergefallen, Runter gefallen, Runter-gefal..."
77,Wegetan,Wehgetan,01-045-2-III-Eis.csv,11,17,11,wehgetan,"[Weh getan, Weh-getan, Weggetan, Angeweht]"
79,Rich,Riech,01-045-2-III-Eis.csv,2,2,2,riech,"[Reich, Siech, Rieche, Riecht, Riecher]"
103,Schtoiper,Stolper,01-057-2-III-Eis.csv,1,1,1,stolper,"[Stolpe, Stolper-, Stopper, Stolpre, Stolpere,..."
221,Cemast,Zermatscht,01-113-2-III-Eis.csv,1,1,1,zermatscht,[Schmatze]
233,Eerber,Erdbeer,01-122-2-III-Eis.csv,1,3,1,Erdbeer,"[Erdbeer-, Erdeber, Erdbeet, Erdbeere, Verderb..."
237,Sis,Sies,01-122-2-III-Eis.csv,2,1,1,sies,"[Siels, Siegs, Siebs, Sees, Siel, Lies, Dies, ..."


In [16]:
display(ct_hun_not_right_TT)
display(ct_hun_right_TT)
display(data_error_target_types.shape[0])

ratio_TT = ct_hun_right_TT/data_error_target_types.shape[0]
print('New upper bound for types is', round(ratio_TT,4)*100,'%')
print(ratio_TT)

376

2583

2959

New upper bound for types is 87.29 %
0.8729300439337614


---

In [17]:
# RECOGNIZED AS RIGHT / NOT RIGHT (false and nor recognized)

# if suggestions is empty, word has been recognized as correct
# <-> if there is a suggestion, words is not recognized as correct

ct_hun_not_right = data_error_types[data_error_types.suggestions.apply(len).gt(0)].shape[0] # check for length greater than 0, i. e. list not empty
ct_hun_right = data_error_types[~data_error_types.suggestions.apply(len).gt(0)].shape[0] # not right is either false or not recognized
display('Hunspell recognizes as not correct...', data_error_types[data_error_types.suggestions.apply(len).gt(0)].head(40))


'Hunspell recognizes as not correct...'

,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_corrected,suggestions
1,Kukt,Kuckt,01-005-2-III-Eis.csv,73,152,73,kuckt,"[Juckt, Duckt, Guckt, Zuckt, Kickt, Kuckuck]"
14,Nahaise,Nachhause,01-006-2-III-Eis.csv,1,10,1,nachhause,"[Nach hause, Nach-hause, Nachschaue, Nachschau]"
28,Weitagegangen,Weitergegangen,01-025-2-III-Eis.csv,1,1,1,weitergegangen,"[Weitergengagen, Weitergegengen, Weiter gegangen, Weiter-gegangen, Weitergegebenen, Weitergetragenen, Weitergetragen, Weitergegeben]"
63,Runtergefalen,Runtergefallen,01-029-2-III-Eis.csv,12,28,12,runtergefallen,"[-untergefallen, Runter gefallen, Runter-gefallen, Heruntergefallen, Herunterfallen, Heruntergefahren, Herunterfalle]"
77,Wegetan,Wehgetan,01-045-2-III-Eis.csv,11,17,11,wehgetan,"[Weh getan, Weh-getan, Weggetan, Angeweht]"
79,Rich,Riech,01-045-2-III-Eis.csv,2,2,2,riech,"[Reich, Siech, Rieche, Riecht, Riecher]"
84,Nachause,Nachhause,01-049-2-III-Eis.csv,1,10,1,nachhause,"[Nach hause, Nach-hause, Nachschaue, Nachschau]"
103,Schtoiper,Stolper,01-057-2-III-Eis.csv,1,1,1,stolper,"[Stolpe, Stolper-, Stopper, Stolpre, Stolpere, Stolpern, Stolpert, Stolzer]"
221,Cemast,Zermatscht,01-113-2-III-Eis.csv,1,1,1,zermatscht,[Schmatze]
233,Eerber,Erdbeer,01-122-2-III-Eis.csv,1,3,1,Erdbeer,"[Erdbeer-, Erdeber, Erdbeet, Erdbeere, Verderber, Beerdige, Beere]"


In [13]:
display(ct_hun_not_right)
display(ct_hun_right)
display(data_error_types.shape[0])

ratio = ct_hun_right/data_error_types.shape[0]
print('New upper bound for types is', round(ratio,4)*100,'%')
print(ratio)

682

8802

9484

New upper bound for types is 92.81 %
0.9280894137494728


### b) Token

In [5]:
ct_hun_not_right_token = data_error_token[data_error_token.suggestions.apply(len).gt(0)].shape[0]
ct_hun_right_token = data_error_token[~data_error_token.suggestions.apply(len).gt(0)].shape[0]
display('Hunspell recognizes as not correct...', data_error_token[data_error_token.suggestions.apply(len).gt(0)].head(40))

display(ct_hun_not_right_token)
display(ct_hun_right_token)
display(data_error_token.shape[0])

ratio_token = ct_hun_right_token/data_error_token.shape[0]
print('New upper bound for tokens is', round(ratio_token,4)*100,'%')
print(ratio_token)

'Hunspell recognizes as not correct...'

,original,corrected,filename,freq_ori,freq_cor,freq_tup,unchanged_corrected,suggestions
1,Kukt,Kuckt,01-005-2-III-Eis.csv,73,152,73,kuckt,"[Juckt, Duckt, Guckt, Zuckt, Kickt, Kuckuck]"
18,Nahaise,Nachhause,01-006-2-III-Eis.csv,1,10,1,nachhause,"[Nach hause, Nach-hause, Nachschaue, Nachschau]"
37,Weitagegangen,Weitergegangen,01-025-2-III-Eis.csv,1,1,1,weitergegangen,"[Weitergengagen, Weitergegengen, Weiter gegang..."
85,Runtergefalen,Runtergefallen,01-029-2-III-Eis.csv,12,28,12,runtergefallen,"[-untergefallen, Runter gefallen, Runter-gefal..."
103,Wegetan,Wehgetan,01-045-2-III-Eis.csv,11,17,11,wehgetan,"[Weh getan, Weh-getan, Weggetan, Angeweht]"
105,Rich,Riech,01-045-2-III-Eis.csv,2,2,2,riech,"[Reich, Siech, Rieche, Riecht, Riecher]"
111,Nachause,Nachhause,01-049-2-III-Eis.csv,1,10,1,nachhause,"[Nach hause, Nach-hause, Nachschaue, Nachschau]"
137,Schtoiper,Stolper,01-057-2-III-Eis.csv,1,1,1,stolper,"[Stolpe, Stolper-, Stopper, Stolpre, Stolpere,..."
179,Runtergefalen,Runtergefallen,01-063-2-III-Eis.csv,12,28,12,runtergefallen,"[-untergefallen, Runter gefallen, Runter-gefal..."
309,Runtergefalen,Runtergefallen,01-108-2-III-Eis.csv,12,28,12,runtergefallen,"[-untergefallen, Runter gefallen, Runter-gefal..."


1404

23197

24601

New upper bound for tokens is 94.28999999999999 %
0.9429291492215763


***

In [9]:
hun_not_right_token = data_error_token[data_error_token.suggestions.apply(len).gt(0)]

In [10]:
hun_not_right_token.to_pickle('df_hunspell_ub_not_right_token_case_insensitive_titlecase.pkl')